In [0]:
gold=spark.table('ecommerce.gold.events')


In [0]:
gold.columns

In [0]:
%sql
WITH daily AS (
  SELECT
    product_id,
    SUM(revenue) AS rev
  FROM ecommerce.gold.events
  GROUP BY product_id
)
SELECT
  product_id,
  rev,
  AVG(rev) OVER (
    ORDER BY product_id
    ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
  ) AS ma7
FROM daily
ORDER BY product_id;


In [0]:
%sql
-- Conversion funnel
SELECT brand,
  SUM(views) as views,
  SUM(purchases) as purchases,
  ROUND(SUM(purchases)*100.0/SUM(views), 2) as conversion_rate
FROM ecommerce.gold.events
GROUP BY brand;

In [0]:
%sql
-- Customer tiers
SELECT
  CASE WHEN cnt >= 10 THEN 'VIP'
       WHEN cnt >= 5 THEN 'Loyal'
       ELSE 'Regular' END as tier,
  COUNT(*) as customers,
  AVG(total_spent) as avg_ltv
FROM (SELECT user_id, COUNT(*) cnt, SUM(price) total_spent
      FROM ecommerce.silver.events WHERE event_type='purchase' GROUP BY user_id)
GROUP BY tier;
